In [56]:
import csv
from datetime import datetime
#from msedge.selenium_tools import Edge, EdgeOptions
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import time
import os
import sys
import requests
from random import randint
from time import sleep
from datetime import datetime

In [11]:
# def get_url(position, location):
#     """Generate url from position and location"""
#     template = 'https://www.indeed.com/jobs?q={}&l={}'
#     position = position.replace(' ', '+')
#     location = location.replace(' ', '+')
#     url = template.format(position, location)
#     return url

In [10]:
# def get_record(card):
#     """Extract job data from single card"""
#     job_title = card.find_element_by_class_name('jobtitle').text
#     company = card.find_element_by_class_name('company').text
#     location = card.find_element_by_class_name('location').text
#     post_date = card.find_element_by_class_name('date').text
#     extract_date = datetime.today().strftime('%Y-%m-%d')
#     summary = card.find_element_by_class_name('summary').text
#     job_url = card.find_element_by_class_name('jobtitle').get_attribute('href')
#     return (job_title, company, location, post_date, extract_date, summary, job_url)

In [9]:
# def get_page_records(cards, job_list, url_set):
#     """Extract all cards from the page"""
#     for card in cards:
#         record = get_record(card)
#         # add if job title exists and not duplicate
#         if record[0] and record[-1] not in url_set:
#             job_list.append(record)
#             url_set.add(record[-1])

In [8]:

# def main(position, location):
#     """Run the main program routine"""
#     scraped_jobs = []
#     scraped_urls = set()
    
#     url = get_url(position, location)
    
#     # setup web driver
#     options = EdgeOptions()
#     options.use_chromium = True
#     driver = Edge(options=options)
#     driver.implicitly_wait(5)
#     driver.get(url)        
    
#     # extract the job data
#     while True:
#         cards = driver.find_elements_by_class_name('jobsearch-SerpJobCard')
#         get_page_records(cards, scraped_jobs, scraped_urls)
#         try:
#             driver.find_element_by_xpath('//a[@aria-label="Next"]').click()
#         except NoSuchElementException:
#             break
#         except ElementNotInteractableException:
#             driver.find_element_by_id('popover-x').click()  # to handle job notification popup
#             get_page_records(cards, scraped_jobs, scraped_urls)
#             continue
    
#     # close driver and save records
#     driver.quit()
#     save_data_to_file(scraped_jobs)

In [7]:
#main('python developer', 'charlotte nc')

In [54]:
# headers = {
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
#     'accept-encoding': 'gzip, deflate, br',
#     'accept-language': 'en-US,en;q=0.9',
#     'cache-control': 'max-age=0',
#     'sec-fetch-dest': 'document',
#     'sec-fetch-mode': 'navigate',
#     'sec-fetch-site': 'none',
#     'sec-fetch-user': '?1',
#     'upgrade-insecure-requests': '1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.67 Safari/537.36 Edg/87.0.664.47'
# }

def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

def get_record(card):
    '''Extract job date from a single record '''
    atag = card.h2.a
    try:
        job_title = atag.get('title')
    except AttributeError:
        job_title = ''
    try:
        company = card.find('span', 'company').text.strip()
    except AttributeError:
        company = ''
    try:
        location = card.find('div', 'recJobLoc').get('data-rc-loc')
    except AttributeError:
        location = ''
    try:
        job_summary = card.find('div', 'summary').text.strip()
    except AttributeError:
        job_summary = ''
    try:
        post_date = card.find('span', 'date').text.strip()
    except AttributeError:
        post_date = ''
    try:
        salary = card.find('span', 'salarytext').text.strip()
    except AttributeError:
        salary = ''
    
    extract_date = datetime.today().strftime('%Y-%m-%d')
    job_url = 'https://www.indeed.com' + atag.get('href')
    
    return (job_title, company, location, job_summary, salary, post_date, extract_date, job_url)

def main(position, location):
    # Run the main program reouting
    records = []  # creating the record list
    url = get_url(position, location)  # create the url while passing in the position and location.
    
    while True:
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')

        for card in cards:
            record = get_record(card)
            records.append(record)

        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
            delay = randint(1, 10)
            sleep(delay)
        except AttributeError:
            break

    with open('results.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['Job Title', 'Company', 'Location', 'Salary', 'Posting Date', 'Extract Date', 'Summary', 'Job Url'])
        writer.writerows(records)


In [57]:
main('data+analyst', 'new+york')

https://www.indeed.com/jobs?q=data+analyst&l=new+york
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=10
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=20
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=30
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=40
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=50
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=60
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=70
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=80
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=90
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=100
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=110
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=120
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=130
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start=140
https://www.indeed.com/jobs?q=data+analyst&l=new+york&start

In [45]:
response = requests.get(test).text

In [47]:
soup = BeautifulSoup(response, 'html.parser')
cards = soup.find_all('div', 'jobsearch-SerpJobCard')

# Functions for indeed job posting data

In [70]:
def get_url(position, location):
        template = 'https://www.indeed.com/jobs?q={}&l={}'
        url = template.format(position, location)
        return url

def get_record(card):
    '''Extract job date from a single record '''
    atag = card.h2.a
    try:
        job_title = atag.get('title')
    except AttributeError:
        job_title = ''
    try:
        company = card.find('span', 'company').text.strip()
    except AttributeError:
        company = ''
    try:
        location = card.find('div', 'recJobLoc').get('data-rc-loc')
    except AttributeError:
        location = ''
    try:
        job_summary = card.find('div', 'summary').text.strip()
    except AttributeError:
        job_summary = ''
    try:
        post_date = card.find('span', 'date').text.strip()
    except AttributeError:
        post_date = ''
    try:
        salary = card.find('span', 'salarytext').text.strip()
    except AttributeError:
        salary = ''
    
    extract_date = datetime.today().strftime('%Y-%m-%d')
    job_url = 'https://www.indeed.com' + atag.get('href')
    
    return (job_title, company, location, job_summary, salary, post_date, extract_date, job_url)

In [85]:
def record_collection(position, location):
    records = []  # creating the record list
    url = get_url(position, location)

    while True:
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'jobsearch-SerpJobCard')

        for card in cards:
            record = get_record(card)
            records.append(record)

        try:
            url = 'https://www.indeed.com' + soup.find('a', {'aria-label': 'Next'}).get('href')
            delay = randint(1, 10)
            sleep(delay)
        except AttributeError:
            break
            
    df = pd.DataFrame(records,columns=['Job Title','Company','Location','Summary','Salary','Post Date','Date Pulled','Job Link'])
    loc = location.replace('+','_')
    df.to_csv(f'{loc}_DataFrame.csv')
    
    return records

In [86]:
record_collection('data+analyst','New+York')

https://www.indeed.com/jobs?q=data+analyst&l=New+York
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=10
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=20
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=30
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=40
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=50
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=60
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=70
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=80
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=90
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=100
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=110
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=120
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=130
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start=140
https://www.indeed.com/jobs?q=data+analyst&l=New+York&start

[('Data Analyst 3 : 278461',
  'CorTech',
  'Tarrytown, NY',
  'Uploads preliminary testing data into database.\nCreates and validates data management report specifications.\nDrives the use of data standards (i.e., CDISC).',
  '',
  '1 day ago',
  '2021-04-27',
  'https://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0ATCZlh4at3dJuJ3v9QYE_c1VOYF6jG6qQshNoY64OlFFfJ6Ge9uDdKamaGhsxPh-9e4wtx9nuTo5DtHkKpDagXZlZ-1N6XYtwDD1RkYsgIIQBj1RykXGk0W1C4fGP4HF_E12AMhJjihpWcMhyRXfNWgMq3-X2dJJCkBCidjXtyehOZvaZ1grThRwCWVeycoRLkr5nVsJa27i_2Kv6EPx0s5_vUZGTx9BYot5DHlzhGs1PxXzjFmxmMI4FAoGgb9kPN6H7SYE8NOOWQ_ZdxUXGDM3rE4yS2qy-ZPusJZcbIfpGpL0FaaDnpecf5MEORZvr-nBO2zfUb1ESnEN1oTLsAYl06d82JLChdflkB0PU6ju-NikNpbTVlwz6Z_468wRr5jEZhxvEwoqL4UtJM9NihidGM9lx3yNK8G0chidCBQOsziS5N-S4CocvP3F-Rgfiyhg29uDDZU_I4P6EnirkV4zmXYf64QwxAhBHveUSxTP3KS_IxYJ55ZUhqR4VE1g399hV_k29rgvsntoT-l9XA_wVJXw5gxPfPU6zss7AjLXohYKSr97A3W2eHvXPJvbjYxBdJuZ7atG7z2VQmOSfPa4m2gupPuKizj9LTlz6Nc-8De2aLtV7Q&p=0&fvj=0&vjs=3'),
 ('Data Analyst (Mid)',
  'Envisi

In [84]:
t = 'Middletown+New+York'
t = t.replace('+','_')


f'{t}_DataFrame.csv'

'Middletown_New_York_DataFrame.csv'

In [68]:
ny_df = pd.DataFrame(records,columns=['Job Title','Company','Location','Summary','Salary','Post Date','Date Pulled','Job Link'])

In [91]:
ny_df

,Job Title,Company,Location,Summary,Salary,Post Date,Date Pulled,Job Link
0,Provider Relations Data Analyst,Hamaspik Choice,"Monsey, NY",Transform data received from various entities ...,,30+ days ago,2021-04-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
1,EDI Analyst 1,"Chapin International, Inc","Batavia, NY",Demonstrate attention to detail and accuracy.\...,,30+ days ago,2021-04-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
2,Business Analyst,Acutis Diagnostics Inc.,"Hicksville, NY",Perform complex data analysis in support of ad...,,30+ days ago,2021-04-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
3,Junior Data Analyst,1-800-Flowers,"Carle Place, NY",Work with analyst team and business teams to d...,,30+ days ago,2021-04-27,https://www.indeed.com/rc/clk?jk=50ea00728c5e8...
4,Data Analyst,Turner,"New York, NY",1-2 years of experience as a data analyst or r...,,5 days ago,2021-04-27,https://www.indeed.com/rc/clk?jk=dcc9f3228dbde...
...,...,...,...,...,...,...,...,...
931,Remote Technical Business Analyst,Insight Global,"New York, NY","At least 5 years of in FinTech, financial serv...",,5 days ago,2021-04-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
932,"Vice President, Business Intelligence Consultant",BlackRock,"New York, NY",5 years or more of reporting development exper...,,18 days ago,2021-04-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
933,Data Analyst,MDLinx,"New York, NY","2+ years of business analysis, data analysis, ...",,30+ days ago,2021-04-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
934,Senior Pharmacy Data Quality Analyst,The Execu|Search Group,"New York, NY","Explain data models, requirements and quality ...",,30+ days ago,2021-04-27,https://www.indeed.com/pagead/clk?mo=r&ad=-6NY...
